In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, pipeline
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('raw.csv').sample(n=10000, ignore_index=True, random_state=42)
data

In [ ]:
data['type_binary'] = data['type'].apply(lambda x: 0 if x == 'benign' else 1)

data_cleaned = data.drop(['Unnamed: 0', 'type'], axis=1)
data_cleaned

In [ ]:
data_cleaned['type_binary'].value_counts()

In [50]:
data['label'] = data_cleaned['type_binary'].astype('category').cat.codes

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [51]:
def prepare_data(data, tokenizer, max_len=256):
    tokens = tokenizer.batch_encode_plus(
        data['url'].tolist(),
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    return {
        'input_ids': tokens['input_ids'],
        'attention_mask': tokens['attention_mask'],
        'labels': torch.tensor(data['label'].values, dtype=torch.long)
    }

train_data_prepared = prepare_data(train_data, tokenizer)
val_data_prepared = prepare_data(val_data, tokenizer)

In [52]:
class URLDataset(Dataset):
    def __init__(self, data):
        self.input_ids = data['input_ids']
        self.attention_mask = data['attention_mask']
        self.labels = data['labels']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

train_dataset = URLDataset(train_data_prepared)
val_dataset = URLDataset(val_data_prepared)

train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32)


In [53]:
# Model initialization
model = AutoModelForCausalLMWithValueHead.from_pretrained('gpt2')
model.config.pad_token_id = model.config.eos_token_id
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# PPO config
ppo_config = PPOConfig(
    model_name='gpt2',
    learning_rate=0.000001,
    batch_size=32,
    forward_batch_size=8,
    mini_batch_size=8,
    gradient_accumulation_steps=4
)

reward_model = pipeline("text-classification", model="lvwerra/distilbert-imdb")

ppo_trainer = PPOTrainer(
    model=model,
    config=ppo_config,
    tokenizer=tokenizer,
    dataset=train_dataset
)

c:\Users\ivomeister\anaconda3\Lib\site-packages\trl\trainer\ppo_config.py:150: UserWarning: Note that using `forward_batch_size` is deprecated, use `mini_batch_size` instead. By setting it you overwrite `mini_batch_size` which affects both the batch size during forward passes and also the mini batch size for PPO optimization.
  warnings.warn(


In [54]:
# Early stopping parameter
best_accuracy = 0

# Generation parameters
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

# Training loop
for epoch in range(10):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to('cuda' if torch.cuda.is_available() else 'cpu')
        attention_mask = batch['attention_mask'].to('cuda' if torch.cuda.is_available() else 'cpu')
        labels = batch['labels'].to('cuda' if torch.cuda.is_available() else 'cpu')

        # Generate responses
        response_tensors = ppo_trainer.generate(input_ids, **generation_kwargs)
        batch['response'] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

        # Compute rewards
        texts = [q + r for q, r in zip(batch['query'], batch['response'])]
        pipe_outputs = reward_model(texts)
        rewards = [torch.tensor(output['score']) for output in pipe_outputs]

        # PPO Step
        stats = ppo_trainer.step(input_ids, response_tensors, rewards)
        total_loss += stats['ppo/loss/total']

    avg_train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_labels = []
    loss_fn = nn.CrossEntropyLoss()

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to('cuda' if torch.cuda.is_available() else 'cpu')
            attention_mask = batch['attention_mask'].to('cuda' if torch.cuda.is_available() else 'cpu')
            labels = batch['labels'].to('cuda' if torch.cuda.is_available() else 'cpu')

            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)
            total_val_loss += loss.item()

            preds = torch.argmax(outputs.logits, dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)

    # Metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)

    print(f'Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}')
    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix - Epoch {epoch+1}')
    plt.show()

    # Early stopping
    if accuracy > best_accuracy:
        best_accuracy = accuracy
    else:
        print("Early stopping activated.")
        break

ValueError: query_tensor must be a tensor of shape (`seq_len`) or a list of tensors of shape (`seq_len`)